In [1]:
from dotenv import load_dotenv
import openai
import os
import json
from pathlib import Path
from dataclasses import asdict

from dataset.ACOS.utils import (
	ACOSQuad, ACOSReview, ACOSPrediction,
	load_acos_reviews, openai_pred_acos, evaluate_acos_preds,
	laptop_acos_train_file, laptop_acos_dev_file,
	restaurant_acos_train_file, restaurant_acos_dev_file,
)

# Load ACOS dataset

In [2]:
lines_to_read = -1
acos_reviews_train: list[ACOSReview] = load_acos_reviews(laptop_acos_train_file, lines_to_read)
acos_reviews_dev: list[ACOSReview] = load_acos_reviews(laptop_acos_dev_file, lines_to_read)

print(f"Loaded {len(acos_reviews_train):,d} training reviews")
print(f"Loaded {len(acos_reviews_dev):,d} dev reviews")

print()
print(f'Review: {acos_reviews_train[0].review}')
print(f'ACOS quads: {acos_reviews_train[0].acos_quads}')


Loaded 2,934 training reviews
Loaded 326 dev reviews

Review: acer wants $ 170 to just look at it then add the repair cost on top of that .
ACOS quads: [ACOSQuad(aspect=['acer'], category='PRICE', opinion=None, sentiment='neutral')]


# ACOS prediction with OpenAI few shots learning

In [3]:
# Set OpenAI keys
load_dotenv()
openai.organization = os.getenv('OPENAI_ORG_ID')
openai.api_key = os.getenv('OPENAI_KEY')

In [4]:
openai_model = "text-davinci-003"
train_data_len = 3

print(f"Using {train_data_len} training examples")
prompt_review_examples: list[ACOSReview] = acos_reviews_train[:train_data_len]
acos_reviews_to_predict: list[ACOSReview] = acos_reviews_dev[:5]
acos_preds: list[ACOSPrediction] = openai_pred_acos(acos_reviews_to_predict, prompt_review_examples, openai_model)


Using 3 training examples
Completed 0 reviews
Total tokens used: 1,177
Credit used: 0.02
Total reviews: 5


In [5]:
print(f'Review: {acos_preds[0].review}')
print(f'ACOS predicted: {acos_preds[0].acos_preds}')
print(f'ACOS targets: {acos_preds[0].acos_targets}')

Review: this unit is ` ` pretty ` ` and stylish , so my high school daughter was attracted to it for that reason .
ACOS predicted: [ACOSQuad(aspect=['unit'], category='DESIGN', opinion=['pretty'], sentiment='positive')]
ACOS targets: [ACOSQuad(aspect=['unit'], category='DESIGN_FEATURES', opinion=['pretty'], sentiment='positive'), ACOSQuad(aspect=['unit'], category='DESIGN_FEATURES', opinion=['stylish'], sentiment='positive')]


In [6]:
precision, recall, f1 = evaluate_acos_preds(acos_preds)

print(f"Precision: {precision:.6f}")
print(f"Recall: {recall:.6f}")
print(f"F1: {f1:.6f}")

Precision: 0.600000
Recall: 0.500000
F1: 0.545455


# Save and load predictions

In [7]:
output_dir = Path('outputs')
output_dir.mkdir(exist_ok=True)

output_file = output_dir / f'preds.jsonl'
with open(output_file, 'w') as f:
    for i, acos_pred in enumerate(acos_preds):
        f.write(json.dumps(asdict(acos_pred)) + '\n')

In [8]:
with open(output_file, 'r') as f:
    l = [ACOSPrediction.from_dict(json.loads(line)) for line in f]

l == acos_preds

True

In [9]:
precision, recall, f1 = evaluate_acos_preds(l)

print(f"Precision: {precision:.6f}")
print(f"Recall: {recall:.6f}")
print(f"F1: {f1:.6f}")

Precision: 0.600000
Recall: 0.500000
F1: 0.545455
